### Анализ игр NBA

Подготовил Григорий Крюков

Источник данных: https://www.kaggle.com/datasets/nathanlauga/nba-games

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_csv("games.csv")

In [3]:
df

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005,Final,1610612748,1610612750,2021,1610612748,104.0,0.398,0.760,...,23.0,53.0,1610612750,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006,Final,1610612741,1610612739,2021,1610612741,101.0,0.443,0.933,...,20.0,46.0,1610612739,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007,Final,1610612759,1610612754,2021,1610612759,108.0,0.412,0.813,...,28.0,52.0,1610612754,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008,Final,1610612744,1610612749,2021,1610612744,122.0,0.484,0.933,...,33.0,55.0,1610612749,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009,Final,1610612743,1610612761,2021,1610612743,115.0,0.551,0.750,...,32.0,39.0,1610612761,127.0,0.471,0.760,0.387,28.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25791,2014-10-06,11400007,Final,1610612737,1610612740,2014,1610612737,93.0,0.419,0.821,...,24.0,50.0,1610612740,87.0,0.366,0.643,0.375,17.0,43.0,1
25792,2014-10-06,11400004,Final,1610612741,1610612764,2014,1610612741,81.0,0.338,0.719,...,18.0,40.0,1610612764,85.0,0.411,0.636,0.267,17.0,47.0,0
25793,2014-10-06,11400005,Final,1610612747,1610612743,2014,1610612747,98.0,0.448,0.682,...,29.0,45.0,1610612743,95.0,0.387,0.659,0.500,19.0,43.0,1
25794,2014-10-05,11400002,Final,1610612761,1610612758,2014,1610612761,99.0,0.440,0.771,...,21.0,30.0,1610612758,94.0,0.469,0.725,0.385,18.0,45.0,1


In [4]:
# ID команд в таблицах дома и на выезде совпадают
sorted(df.HOME_TEAM_ID.unique()) == sorted(df.VISITOR_TEAM_ID.unique())

True

In [5]:
team_id = sorted(df.HOME_TEAM_ID.unique())
len(team_id)

30

In [6]:
N = 30
prev_win = [1 for i in range(N)] # Выиграна ли предыдущая игра
TM = [np.zeros((2, 2)) for i in range(N)] # Матрицы переходов
k = 1610612737 # Параметр, вычитание которого из id команды приводит к порядковому номеру
# Проходимся циклом с конца таблицы (в хронологическом порядке), заполняем соответствующие значения матриц переходов
for i in range(len(df) - 1, -1, -1):
    home = df.HOME_TEAM_ID[i] - k
    visitor = df.VISITOR_TEAM_ID[i] - k
    if df.HOME_TEAM_WINS[i] == 1:
        TM[home][1 - prev_win[home]][0] += 1
        TM[visitor][1 - prev_win[visitor]][1] += 1
        prev_win[home] = 1
        prev_win[visitor] = 0
    else:
        TM[home][1 - prev_win[home]][1] += 1
        TM[visitor][1 - prev_win[visitor]][0] += 1
        prev_win[home] = 0
        prev_win[visitor] = 1

In [7]:
# Промежуточная проверка на правильность: сумма всех значений в матрицах переходов в два раза больше числа игр
S = 0
for i in range(N):
    S += TM[i].sum()
assert S == 2 * len(df) 
# Для каждой из команд модуль разницы между числом побед после поражения и числом поражений после побед не может превосходить 1
for i in range(N):
    assert np.abs(TM[i][0][1] - TM[i][1][0]) <= 1

In [8]:
# Теперь нам осталось получить "настоящие" оценки матриц переходов
# Для этого все строки матриц TM нормируем так, чтобы сумма по строкам была равна 1
for i in range(N):
    TM[i][0] /= TM[i][0].sum()
    TM[i][1] /= TM[i][1].sum()

In [9]:
# Полученные матрицы переходов
TM

[array([[0.488228  , 0.511772  ],
        [0.44745395, 0.55254605]]),
 array([[0.59643917, 0.40356083],
        [0.50809465, 0.49190535]]),
 array([[0.58555556, 0.41444444],
        [0.43155452, 0.56844548]]),
 array([[0.4816273 , 0.5183727 ],
        [0.43439912, 0.56560088]]),
 array([[0.51847575, 0.48152425],
        [0.49116608, 0.50883392]]),
 array([[0.5748503 , 0.4251497 ],
        [0.56648936, 0.43351064]]),
 array([[0.57525773, 0.42474227],
        [0.53446034, 0.46553966]]),
 array([[0.59816887, 0.40183113],
        [0.5156658 , 0.4843342 ]]),
 array([[0.59146341, 0.40853659],
        [0.52624672, 0.47375328]]),
 array([[0.54983203, 0.45016797],
        [0.479092  , 0.520908  ]]),
 array([[0.58665231, 0.41334769],
        [0.44859813, 0.55140187]]),
 array([[0.57803468, 0.42196532],
        [0.55176768, 0.44823232]]),
 array([[0.4969475 , 0.5030525 ],
        [0.45870536, 0.54129464]]),
 array([[0.4222561 , 0.5777439 ],
        [0.38594705, 0.61405295]]),
 array([[0.48138298,

In [10]:
is_win_lover = 0
for i in range(N):
    if TM[i][0][0] > TM[i][1][0]:
        is_win_lover += 1
is_win_lover

29

In [11]:
win_lose_diff = 0
for i in range(N):
    win_lose_diff += TM[i][0][0] - TM[i][1][0]
win_lose_diff / N * 100

6.22477720214843

Для 29 команд из 30 вероятность выиграть после победы больше, чем вероятность выиграть после поражения.

В среднем вероятность победы в текущей игре при победе в прошлой игре на 6.22% выше, чем вероятность победы в текущей игре при поражении в прошлой игре.

Это подтверждает состоятельность нашего предположения о психологических эффектах.

#### Силы команд

Базовый подход: доля выигранных матчей.

In [30]:
w = np.zeros(N)
prev_win = [1 for i in range(N)] # Выиграна ли предыдущая игра
A = [np.zeros((2, 2)) for i in range(N)] # Матрицы переходов
k = 1610612737 # Параметр, вычитание которого из id команды приводит к порядковому номеру
# Проходимся циклом с конца таблицы (в хронологическом порядке), заполняем соответствующие значения матриц переходов
for i in range(len(df) - 1, -1, -1):
    home = df.HOME_TEAM_ID[i] - k
    visitor = df.VISITOR_TEAM_ID[i] - k
    if df.HOME_TEAM_WINS[i] == 1:
        A[home][1 - prev_win[home]][0] += 1
        A[visitor][1 - prev_win[visitor]][1] += 1
        prev_win[home] = 1
        prev_win[visitor] = 0
    else:
        A[home][1 - prev_win[home]][1] += 1
        A[visitor][1 - prev_win[visitor]][0] += 1
        prev_win[home] = 0
        prev_win[visitor] = 1
        
# Теперь поделим число выигранных матчей на число всех матчей
for i in range(N):
    w[i] = A[i].sum(axis=1)[0] / A[i].sum()

In [31]:
w

array([0.46647399, 0.55733186, 0.5107832 , 0.45656081, 0.50495627,
       0.57126568, 0.55779183, 0.56203545, 0.56357388, 0.51618497,
       0.52044818, 0.56721311, 0.47755102, 0.4004884 , 0.44183314,
       0.40256567, 0.44791054, 0.52036718, 0.45147929, 0.51665693,
       0.49853372, 0.41005451, 0.64105379, 0.52561888, 0.51515152,
       0.53962704, 0.49853544, 0.43506494, 0.48242075, 0.40103159])

In [35]:
# Сортируем команды от слабейшей к сильнейшей
np.argsort(w)

array([13, 29, 15, 21, 27, 14, 16, 18,  3,  0, 12, 28, 20, 26,  4,  2, 24,
        9, 19, 17, 10, 23, 25,  1,  6,  7,  8, 11,  5, 22], dtype=int64)

Стационарное распределение матрицы.

In [14]:
def get_stat(p, q):
    return q/(1-p+q), (1-p)/(1-p+q)

In [15]:
get_stat(0.5, 0.5)

(0.5, 0.5)

In [16]:
get_stat(0.7, 0.5)

(0.625, 0.37500000000000006)

In [18]:
get_stat(0.5, 0.3)

(0.37499999999999994, 0.625)

In [20]:
# Проверка на стабильность результатов в зависимости от нумерации команд:
for i in range(1000):
    p = np.random.random()
    q = np.random.random()
    assert np.round(get_stat(p, q)[0] - get_stat(1-q, 1-p)[1], 4) == 0

Теперь посчитаем полученную метрику для модели с психологическим эффектом.

$$u(i) = \frac{p_i + q_i - 1}{p_i q_i}$$

In [38]:
def func_u(p, q):
    return (p + q - 1) / (p * q)

u = np.zeros(N)
for i in range(N):
    p = TM[i][0][0]
    q = TM[i][1][0]
    u[i] = func_u(p, q)

In [39]:
u

array([-0.29441624,  0.34494196,  0.06770929, -0.40136793,  0.03786192,
        0.43402778,  0.35686192,  0.36904762,  0.37817827,  0.10980187,
        0.13394495,  0.40697941, -0.19454557, -1.17689531, -0.54696133,
       -1.09048886, -0.45501285,  0.15106383, -0.44489084,  0.11014447,
       -0.01138952, -1.06195773,  0.66452462,  0.16678039,  0.10655738,
        0.26719057, -0.01096491, -0.67380952, -0.14102542, -1.10431655])

In [41]:
# Сортируем команды от слабейшей к сильнейшей
np.argsort(u)

array([13, 29, 15, 21, 27, 14, 16, 18,  3,  0, 12, 28, 20, 26,  4,  2, 24,
        9, 19, 10, 17, 23, 25,  1,  6,  7,  8, 11,  5, 22], dtype=int64)

Сравним сортировки w_i, u_i.

In [42]:
np.argsort(w), np.argsort(u)

(array([13, 29, 15, 21, 27, 14, 16, 18,  3,  0, 12, 28, 20, 26,  4,  2, 24,
         9, 19, 17, 10, 23, 25,  1,  6,  7,  8, 11,  5, 22], dtype=int64),
 array([13, 29, 15, 21, 27, 14, 16, 18,  3,  0, 12, 28, 20, 26,  4,  2, 24,
         9, 19, 10, 17, 23, 25,  1,  6,  7,  8, 11,  5, 22], dtype=int64))

In [43]:
np.corrcoef(np.argsort(w), np.argsort(u))

array([[1.      , 0.978198],
       [0.978198, 1.      ]])